In [1]:
# Import required libraries
import os
from langchain.vectorstores import Vectara
from langchain.chat_models.openai import ChatOpenAI
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

In [2]:
# config settings for OpenAI and Vectra
customer_id = '189024573'
corpus_id = '2'
api_key = 'PROVIDE VECTRA API KEY OR ACCESS THROUGH CONFIG FILE'
openai_key = 'PROVIDE OPENAI API KEY OR ACCESS THROUGH CONFIG FILE'
os.environ["VECTARA_CUSTOMER_ID"] = customer_id
os.environ["VECTARA_CORPUS_ID"] = corpus_id
os.environ["VECTARA_API_KEY"] = api_key
os.environ["OPENAI_API_KEY"] = openai_key

In [3]:
# Defining the Retriever
vectara = Vectara()
summary_config = {
    "is_enabled": True, "max_results": 50,
    "response_lang": "en",
    "prompt_name": "Generate MCQ"
}
retriever = vectara.as_retriever(
    search_kwargs={"k": 100, "summary_config": summary_config}
)

In [6]:
### Define the model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

### Contextualize question ###
contextualize_q_system_prompt = """
Given a chat history and the user topic \
which might reference context in the chat history, formulate a MCQ question. \
which can be understood without the chat history. Do NOT answer the question, \
Recreate the question based on the retrieved documents with out ging same question as it is.
"""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
qa_system_prompt = """
You are an assistant for creating MCQ questions based on the context for user. \
Don not display exactly same question, shufle the options and recreate the question. \ 
Use the following pieces of retrieved context to formulate the MCQ question. \
Don't repeat the question. \
Display one MCQ question and options only at a time. \
Question in one line and options in separate line and wait for the user Answer. \
Apply adaptive ranking by evaluating the retrieved MCQs, re-ranking them based on difficulty, and sequencing the questions according to user responses.\ 
If the user answers correctly, respond with 'Your answer is correct' and, ask would you like to try another question from same topic (Yes/exit).\
If the user answers incorrectly, respond with 'Your answer is wrong' and provide the correct answer with a brief explanation and, ask would you like to try another question from same topic (Yes/exit).\.\
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [7]:
## Calling the model iteratively to give user inputs for model
while True:
    user_input = input("Enter the response: ")
    if user_input.lower() == "exit":
        print("bye!")
        break
    print(conversational_rag_chain.invoke( {"input": user_input}, config={"configurable": {"session_id": "abc123"}},)["answer"])

Enter the response:  animals


What type of animals are found in the phylum Chordata and have a backbone?

A: invertebrates
B: vertebrates
C: arthropods
D: mollusks


Enter the response:  B


Your answer is correct! Would you like to try another question from the same topic? (Yes/exit)


Enter the response:  Yes


What is the largest mammal on earth?

A: giraffe
B: tiger shark
C: water buffalo
D: blue whale


Enter the response:  A


Your answer is incorrect. The correct answer is D: blue whale. Would you like to try another question from the same topic? (Yes/exit)


Enter the response:  exit


bye!
